# Data Preparation

## Introduction

**Context & Motivation**

Vaccination is a cornerstone of public health, contributing significantly to the prevention of disease and the reduction of child mortality worldwide. Despite its proven effectiveness, global access to and trust in vaccines remains uneven — influenced by political, economic, and cultural factors. This project explores these dynamics from a data-driven perspective.

**Objectives**
1. Read data for various sources (UNICEF, World Bank, World Values Survey, V Dem)
2. Handle missing data (or exclude dataset if needed)
4. Ensure consistency (country names, data format, etc.)
5. Merge data for final dataset 

The final dataset will include the following variables:
1. **country** - name of countries
2. **region** - region name - USND
3. **subregion** - sub-region name - USND
4. **year** - applicable year
5. **vac_index** - index of vaccination calculated as average of % ofvaccinated population with globally recommended vaccines (BCG, DTP3, POL3, IPV2, MCV2, RCV1, HEPB3, HEPBB, HIB3, PCV3, ROTAC) - UNICEF
6. **gpd** - gdp per capita (US$) - World Bank
7. **health_exp** - health expenditure (% of GDP) - World Bank
8. **child_mort** - under 5 years old children mortality (per 1000 live births) - World Bank
9. **internet_use** - % of population using internet - World Bank
10. **gov_trust** - % of population trusting government (yes/no) - UNICEF
11. **polarization** - score based on expert assessments of societal divisions - V-Dem

Separated DataSets will be used in relational PowerBI database, whereas merged dataframe in exploratory analysis and predictions.

In [1]:
#importing libraries
import pandas as pd
import numpy as np

## Vaccinations - Target Variable

Vaccinations Excel includes UNICEF data on % of population being vaccinated in a given year. Each vaccine data is included in a separate tab. As some vaccines are only recommended for a few regions, there will be excluded from the analysis for a consistency.

### All Vaccines DataFrame

In [2]:
#reading the file to get sheet names
vaccines = pd.ExcelFile('01_Vaccinations.xlsx')

#listing sheet names
vaccines.sheet_names

['ReadMe',
 'BCG',
 'DTP1',
 'DTP3',
 'HEPB3',
 'HEPBB',
 'HIB3',
 'IPV1',
 'IPV2',
 'MCV1',
 'MCV2',
 'MENGA',
 'PCV3',
 'POL3',
 'RCV1',
 'ROTAC',
 'YFV',
 'regional_global']

In [3]:
# Read in each sheet for vaccines to be included in the analysis
sheet_names = ['BCG' , 'DTP3', 'POL3', 'IPV2', 'MCV2', 'RCV1', 'HEPB3', 'HEPBB', 'HIB3', 'PCV3', 'ROTAC']
sheets = [vaccines.parse(name) for name in sheet_names] 

# Combine them into one dataframe
vac_df = pd.concat(sheets, ignore_index=True)
vac_df.head()

,unicef_region,iso3,country,vaccine,2023,2022,2021,2020,2019,2018,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,ROSA,AFG,Afghanistan,BCG,68.0,69.0,65.0,72.0,74.0,82.0,...,64.0,66.0,60.0,60.0,57.0,51.0,44.0,46.0,43.0,30.0
1,ECAR,ALB,Albania,BCG,99.0,99.0,99.0,98.0,99.0,99.0,...,97.0,99.0,98.0,97.0,98.0,97.0,95.0,94.0,93.0,93.0
2,MENA,DZA,Algeria,BCG,99.0,98.0,98.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,98.0,98.0,98.0,98.0,97.0,97.0
3,ESAR,AGO,Angola,BCG,73.0,60.0,56.0,58.0,69.0,72.0,...,70.0,73.0,75.0,54.0,51.0,63.0,54.0,76.0,70.0,53.0
4,LACR,ARG,Argentina,BCG,69.0,81.0,80.0,75.0,85.0,93.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,95.0,92.0,95.0


In [4]:
#choosing applicable columns and reordering
vac_df = vac_df.iloc[:, [2, 1, 3, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5]]


In [5]:
#getting number of rows and columns
vac_df.shape

(1782, 14)

In [6]:
#listing datatypes
vac_df.dtypes

country     object
iso3        object
vaccine     object
2012       float64
2013       float64
2014       float64
2015       float64
2016       float64
2017       float64
2018       float64
2019       float64
2020       float64
2021       float64
2022       float64
dtype: object

###  Vaccination Index

As data on specific vaccines could be possibly used in the analysis, there will be new dataframe created for vaccination index - mean score of vaccinations for included vaccines. 

In [7]:
#removing vaccine type column
v_index_df = vac_df.drop(['vaccine'], axis=1)

#calculating vaccination index 
v_index_df = v_index_df.groupby(['country', 'iso3']).mean().round(2)
v_index_df.reset_index(inplace=True)
v_index_df.head()

,country,iso3,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,62.50,60.83,52.75,56.25,57.12,57.75,60.44,59.33,58.44,51.80,56.40
1,Albania,ALB,98.33,99.00,98.33,98.67,98.00,98.56,98.00,97.89,96.40,95.82,95.27
2,Algeria,DZA,96.14,95.86,96.71,96.71,89.88,92.33,90.00,88.11,85.67,83.44,81.00
3,Andorra,AND,96.29,94.86,94.88,94.38,95.12,98.00,97.57,97.57,97.29,97.71,96.71
4,Angola,AGO,56.80,48.33,50.71,51.75,48.75,50.25,55.78,54.56,47.89,40.00,36.50


In [8]:
#checking number of countries in index df
v_index_df.shape

(195, 13)

In [9]:
#checking missing values
v_index_df.isna().sum()

country    0
iso3       0
2012       0
2013       0
2014       0
2015       0
2016       0
2017       0
2018       0
2019       0
2020       0
2021       0
2022       0
dtype: int64

In [10]:
#listing countries with long names (longer than 15 characters)
long_country_names = v_index_df[v_index_df['country'].str.len() > 15]
long_country_names

,country,iso3,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
5,Antigua and Barbuda,ATG,96.50,96.33,97.50,93.00,86.50,95.50,95.00,93.50,92.33,80.12,88.75
20,Bolivia (Plurinational State of),BOL,93.43,87.86,83.00,90.50,89.00,84.75,79.89,72.89,68.44,70.10,68.20
21,Bosnia and Herzegovina,BIH,91.71,90.29,88.43,83.14,78.14,77.29,75.29,74.43,73.00,74.29,73.86
24,Brunei Darussalam,BRN,95.86,97.43,97.86,98.29,98.57,98.57,98.86,98.57,96.50,96.88,96.75
32,Central African Republic,CAF,51.50,25.33,49.67,48.00,47.00,45.50,45.50,45.50,45.50,45.50,45.50
46,Democratic People's Republic of Korea,PRK,85.14,93.00,93.00,95.14,97.14,97.71,97.57,97.29,94.62,63.38,27.00
47,Democratic Republic of the Congo,COD,64.67,67.17,69.83,70.33,70.83,73.00,73.50,64.71,65.86,63.14,65.86
51,Dominican Republic,DOM,73.00,81.12,82.11,78.00,76.89,81.67,78.50,83.00,76.20,80.20,77.36
55,Equatorial Guinea,GNQ,63.33,64.00,59.80,46.80,59.80,62.40,64.60,67.20,68.80,62.33,63.50
80,Iran (Islamic Republic of),IRN,98.00,98.29,98.57,98.00,98.38,98.38,98.38,98.38,98.25,98.00,98.44


In [11]:
#shortening the names
v_index_df['country'] = v_index_df['country'].replace(
    {"Bolivia (Plurinational State of)": "Bolivia", 
     "Democratic People's Republic of Korea": "North Korea",
     "Democratic Republic of the Congo": "Congo",
     "Iran (Islamic Republic of)": "Iran",
     "Lao People's Democratic Republic": "Laos",
     "Micronesia (Federated States of)": "Micronesia",
     "Netherlands (Kingdom of the)": "Netherlands",
     "Republic of Korea": "South Korea",
     "Republic of Moldova": "Moldova",
     "Russian Federation": "Russia",
     "State of Palestine": "Palestine",
     "Syrian Arab Republic": "Syria",
     "United Republic of Tanzania": "Tanzania",
     "Venezuela (Bolivarian Republic of)": "Venezuela"}
)

### ISO codes

In [12]:
# creating a list of iso codes for target values for other values clean up 
# (to choose right missing values method handling) 
vac_iso_codes = v_index_df["iso3"]

### Countries map

In [13]:
#creating a map for countries names for consistency
countries_map = v_index_df[["country", "iso3"]]

## Socio-Economic Variables

Data in this section comes from World Bank and have analogical structure.

### GDP

In [14]:
#reading the file
gdp_df = pd.read_csv('02_gdp.csv', skiprows=4)
gdp_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,27441.529662,28440.051964,30082.127645,31096.205074,22855.932320,27200.061079,30559.533535,33984.790620,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,186.132432,186.947182,197.408105,225.447007,209.005786,226.883067,...,1329.777824,1520.171298,1538.924188,1493.780445,1344.080962,1522.590088,1628.024526,1659.515290,NaN,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,415.707417,NaN,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,121.938353,127.452629,133.825452,139.006714,148.547736,155.563837,...,1616.843198,1560.162999,1703.896392,1783.654365,1664.249176,1747.840549,1777.235012,1568.724310,NaN,NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1807.952941,2437.259712,2538.591391,2189.855714,1449.922867,1925.874661,2929.694455,2308.159767,NaN,NaN


In [15]:
#choosing applicable columns 
gdp_df = gdp_df[['Country Name', 'Country Code', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']]

#filtering for countries included in vac_index df
gdp_df = gdp_df[gdp_df["Country Code"].isin(vac_iso_codes)]

In [16]:
#checking missing values
gdp_df.isna().sum()

Country Name    0
Country Code    0
2012            2
2013            2
2014            2
2015            3
2016            4
2017            4
2018            4
2019            4
2020            4
2021            5
2022            5
dtype: int64

In [17]:
#checking number of rows and columns
gdp_df.shape

(193, 13)

In [18]:
#removing records with at least 4 missing values 
gdp_df = gdp_df.dropna(thresh=gdp_df.shape[1]-4)

#filling in missing values with linear interpolation
gdp_df.iloc[:, 2:] = gdp_df.iloc[:, 2:].interpolate(method='linear', axis=1)

#dropping records with any missing values left
gdp_df = gdp_df.dropna()

gdp_df.shape

(189, 13)

In [19]:
gdp_df = gdp_df.round(2)

In [20]:
# Merge gdp_df with countries_map based on the ISO code
gdp_df = gdp_df.merge(countries_map, left_on='Country Code', right_on='iso3', how='left')

# Reorder columns to have: country, iso3, 2012, 2013, ...
cols = ['country', 'iso3'] + [col for col in gdp_df.columns if col.startswith('20')]
gdp_df = gdp_df[cols]
gdp_df.head()

,country,iso3,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,651.42,637.09,625.05,565.57,522.08,525.47,491.34,496.60,510.79,356.50,357.26
1,Angola,AGO,5086.03,5057.75,5006.00,3213.90,1807.95,2437.26,2538.59,2189.86,1449.92,1925.87,2929.69
2,Albania,ALB,4247.63,4413.06,4578.63,3952.80,4124.06,4531.03,5365.49,5460.43,5370.78,6413.28,6846.43
3,Andorra,AND,41500.54,42470.32,44369.66,38654.93,40129.82,40672.99,42819.77,41257.80,37361.09,42425.70,42414.06
4,United Arab Emirates,ARE,52034.48,52023.52,51381.41,43535.00,41326.40,42340.59,45689.86,44251.45,37173.88,43360.02,49899.07


The rest of data from World Bank was cleaned analogically, so the following functions were created:

In [21]:
def read_and_check_quality(filepath):
    """
    - Reads the file skipping first 4 rows
    - Chooses columns for country name and code, and years 2012-2022
    - filters for countris included in vaccinations scope

    Parameters:
        filepath: The input filepath

    Returns:
        pd.DataFrame: Filtred dataframe
    """
    
    #reading the file
    df = pd.read_csv(filepath, skiprows=4)
    
    #choosing applicable columns 
    df = df[['Country Name', 'Country Code', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']]
    
    #filtering for countries included in vac_index df
    df = df[df["Country Code"].isin(vac_iso_codes)]

    return df

In [22]:
def clean_and_merge(df, countries_map, code_column='Country Code'):
    """
    Cleans the dataframe by:
    - Removing rows with at least 4 missing values
    - Interpolating remaining missing values
    - Dropping any rows still containing NaNs
    - Rounding numeric values
    - Merging with countries_map on ISO code
    - Reordering columns to: country, iso3, <year columns>

    Parameters:
        df (pd.DataFrame): The input dataframe
        countries_map (pd.DataFrame): Mapping dataframe with 'iso3' and 'country'
        code_column (str): Column name in df with ISO3 codes to join on (default: 'Country Code')

    Returns:
        pd.DataFrame: Cleaned and reordered dataframe
    """
    # Drop rows with at least 4 missing values
    df = df.dropna(thresh=df.shape[1] - 4)

    # Interpolate missing values
    df.iloc[:, 2:] = df.iloc[:, 2:].interpolate(method='linear', axis=1)

    # Drop any remaining NaNs
    df = df.dropna()

    # Round to 2 decimal places
    df = df.round(2)

    # Merge with country mapping
    df = df.merge(countries_map, left_on='Country Code', right_on='iso3', how='left')

    # Reorder columns
    year_cols = [col for col in df.columns if col.startswith('20')]
    df = df[['country', 'iso3'] + year_cols]

    return df


### Gini Index

In [23]:
#calling reading function
gini_df = read_and_check_quality('03_gini.csv')

#checking missing values
gini_df.isna().sum()

Country Name      0
Country Code      0
2012            110
2013            119
2014            112
2015            109
2016            113
2017            116
2018            102
2019            116
2020            128
2021            122
2022            165
dtype: int64

In [24]:
gini_df.shape

(193, 13)

In [25]:
#calling the cleaning function
gini_df = clean_and_merge(gini_df, countries_map)
gini_df.shape

(62, 13)

Gini index to exclude from the analysis

### Health Expedinture

In [26]:
#calling reading function
health_df = read_and_check_quality('04_health.csv')

#checking missing values
health_df.isna().sum()

Country Name    0
Country Code    0
2012            4
2013            3
2014            3
2015            3
2016            3
2017            2
2018            1
2019            1
2020            1
2021            1
2022            2
dtype: int64

In [27]:
health_df.shape

(193, 13)

In [28]:
#calling the cleaning function
health_df = clean_and_merge(health_df, countries_map)
health_df.shape

(189, 13)

In [29]:
health_df.head()

,country,iso3,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,7.90,8.81,9.53,10.11,11.82,12.62,14.21,14.83,15.53,21.51,23.09
1,Angola,AGO,2.40,2.73,2.43,2.61,2.71,2.79,2.59,2.48,3.27,3.09,2.93
2,Albania,ALB,6.16,6.28,6.43,6.46,6.73,6.58,6.67,6.86,7.50,7.40,6.19
3,Andorra,AND,6.71,6.80,6.77,6.92,6.91,7.07,7.38,7.32,8.79,8.33,7.54
4,United Arab Emirates,ARE,3.34,3.50,3.53,3.47,3.90,4.01,4.10,4.40,5.82,5.30,4.68


### Children Mortality

In [30]:
#calling reading function
child_mort_df = read_and_check_quality('05_child_mortality.csv')

#checking missing values
child_mort_df.isna().sum()

Country Name    0
Country Code    0
2012            0
2013            0
2014            0
2015            0
2016            0
2017            0
2018            0
2019            0
2020            0
2021            0
2022            0
dtype: int64

In [31]:
child_mort_df.shape

(193, 13)

In [32]:
#there are no missing values but will apply finction for merge
child_mort_df = clean_and_merge(child_mort_df, countries_map)
child_mort_df.head()

,country,iso3,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,81.2,78.0,75.1,72.4,70.0,67.6,65.4,63.3,61.3,59.3,57.4
1,Angola,AGO,104.8,98.3,92.8,88.2,84.1,80.5,77.3,74.4,71.6,69.0,66.4
2,Albania,ALB,11.2,10.5,9.9,9.6,9.4,9.3,9.3,9.4,9.4,9.5,9.4
3,Andorra,AND,4.1,3.9,3.7,3.5,3.4,3.2,3.1,3.0,2.9,2.8,2.7
4,United Arab Emirates,ARE,8.4,8.2,7.9,7.7,7.3,7.0,6.7,6.3,6.0,5.6,5.3


### Internet Use

In [33]:
#calling reading function
internet_df = read_and_check_quality('08_internet.csv')

#checking missing values
internet_df.isna().sum()

Country Name     0
Country Code     0
2012             6
2013             6
2014             5
2015             6
2016             5
2017             4
2018            21
2019            11
2020            10
2021            16
2022            15
dtype: int64

In [34]:
internet_df.shape

(193, 13)

In [35]:
#calling the cleaning function
internet_df = clean_and_merge(internet_df, countries_map)
internet_df.shape

(183, 13)

In [36]:
internet_df.head()

,country,iso3,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,5.45,5.9,7.0,8.26,11.0,13.5,16.80,17.6,17.0,16.5,17.2
1,Angola,AGO,7.70,13.0,21.4,22.00,23.2,26.0,29.00,32.1,36.6,39.4,42.1
2,Albania,ALB,49.40,51.8,54.3,56.90,59.6,62.4,65.40,68.6,72.2,79.3,82.6
3,Andorra,AND,82.70,84.4,86.1,87.90,89.7,91.6,91.15,90.7,93.2,93.9,94.5
4,United Arab Emirates,ARE,85.00,88.0,90.4,90.50,90.6,94.8,98.50,99.2,100.0,100.0,100.0


### Government Trust

Data in this section comes from OECD trust survey.

In [37]:
#reading data
trust = pd.read_csv('06_gov_trust_oecd.csv')
trust.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,MEASURE,Measure,UNIT_MEASURE,Unit of measure,...,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals,BASE_PER,Base period
0,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_FWB(1.1),Future well-being,I,CZE,Czechia,14_3,Trust in government,PT_POP_Y_GE15,Percentage of population aged 15 years or over,...,44.579404,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
1,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_FWB(1.1),Future well-being,I,CZE,Czechia,14_3,Trust in government,PT_POP_Y_GE15,Percentage of population aged 15 years or over,...,44.579404,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
2,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_FWB(1.1),Future well-being,I,CZE,Czechia,14_3,Trust in government,PT_POP_Y_GE15,Percentage of population aged 15 years or over,...,44.579404,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
3,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_FWB(1.1),Future well-being,I,CZE,Czechia,14_3,Trust in government,PT_POP_Y_GE15,Percentage of population aged 15 years or over,...,22.121649,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
4,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_FWB(1.1),Future well-being,I,CZE,Czechia,14_3,Trust in government,PT_POP_Y_GE15,Percentage of population aged 15 years or over,...,22.121649,NaN,A,Normal value,0,Units,2,Two,NaN,NaN


In [38]:
#listing columns in a dataframe
trust.columns

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA',
       'Reference area', 'MEASURE', 'Measure', 'UNIT_MEASURE',
       'Unit of measure', 'AGE', 'Age', 'SEX', 'Sex', 'EDUCATION_LEV',
       'Education level', 'DOMAIN', 'Domain', 'TIME_PERIOD', 'Time period',
       'OBS_VALUE', 'Observation value', 'OBS_STATUS', 'Observation status',
       'UNIT_MULT', 'Unit multiplier', 'DECIMALS', 'Decimals', 'BASE_PER',
       'Base period'],
      dtype='object')

In [39]:
#filtering education level for total
trust = trust[trust['Education level'] == 'Total']

In [40]:
#choosing applicable columns
trust = trust[['REF_AREA','TIME_PERIOD', 'OBS_VALUE','BASE_PER','Base period']]
trust.head()

,REF_AREA,TIME_PERIOD,OBS_VALUE,BASE_PER,Base period
0,CZE,2016,44.579404,NaN,NaN
1,CZE,2015,44.579404,NaN,NaN
2,CZE,2014,44.579404,NaN,NaN
3,CZE,2013,22.121649,NaN,NaN
4,CZE,2012,22.121649,NaN,NaN


In [41]:
#checking shape
trust.shape

(449, 5)

In [42]:
#removing rows with empty values and checking how many were romoved
trust = trust.dropna(axis=1)
trust.shape

(449, 3)

There are no NaNs

In [43]:
#renaming columns
trust_df =trust.set_axis(['iso3', 'year', 'gov_trust'], axis=1)
trust_df.head()

,iso3,year,gov_trust
0,CZE,2016,44.579404
1,CZE,2015,44.579404
2,CZE,2014,44.579404
3,CZE,2013,22.121649
4,CZE,2012,22.121649


In [44]:
#checking datatypes
trust_df['gov_trust'] = trust_df['gov_trust'].round(2)
trust_df.dtypes

iso3          object
year           int64
gov_trust    float64
dtype: object

In [45]:
#filtering for countries included in vaccination data to check the number of
#applicable countries
trust_df = trust_df[trust_df["iso3"].isin(vac_iso_codes)]
trust_df[['iso3']].nunique()

iso3    38
dtype: int64

Data only for OECD countries is available.

In [46]:
#getting standardized country name 
trust_df = trust_df.merge(countries_map, on='iso3', how='left')

#reordering
trust_df = trust_df[['country', 'iso3', 'year', 'gov_trust']]
trust_df.head()

,country,iso3,year,gov_trust
0,Czechia,CZE,2016,44.58
1,Czechia,CZE,2015,44.58
2,Czechia,CZE,2014,44.58
3,Czechia,CZE,2013,22.12
4,Czechia,CZE,2012,22.12


### Polarization

Data in this section comes from V_Dem (Varieties of Democracy) project and includes multiple variables measuring democracy. For the immunization project, data on polarization will be extracted.

In [47]:
pol = pd.read_csv('07_v_dem.csv')
pol.head()

,country_name,country_text_id,country_id,year,historical_date,project,historical,histname,codingstart,codingend,...,v2xex_elecleg,v2xps_party,v2xps_party_codelow,v2xps_party_codehigh,v2x_divparctrl,v2x_feduni,v2xca_academ,v2xca_academ_codelow,v2xca_academ_codehigh,v2xca_academ_sd
0,Mexico,MEX,3,1789,1789-12-31,1,1,Viceroyalty of New Spain,1789,2024,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,Mexico,MEX,3,1790,1790-12-31,1,1,Viceroyalty of New Spain,1789,2024,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,Mexico,MEX,3,1791,1791-12-31,1,1,Viceroyalty of New Spain,1789,2024,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,Mexico,MEX,3,1792,1792-12-31,1,1,Viceroyalty of New Spain,1789,2024,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,Mexico,MEX,3,1793,1793-12-31,1,1,Viceroyalty of New Spain,1789,2024,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [48]:
# Selecting relevant columns
pol_df = pol[['country_text_id', 'year', 'v2pepwrsoc']]

In [49]:
# Filter data for years 2012 to 2022
pol_df = pol_df[(pol_df['year'] >= 2012) & (pol_df['year'] <= 2022)]

#changing column names
pol_df = pol_df.set_axis(['iso3', 'year', 'polarization'], axis=1)

#rounding values to 2 decimals 
pol_df['polarization'] = pol_df['polarization'].round(2)
pol_df.head()

,iso3,year,polarization
223,MEX,2012,1.0
224,MEX,2013,1.0
225,MEX,2014,1.0
226,MEX,2015,1.0
227,MEX,2016,1.0


In [50]:
#filtering data with vaccination countries
pol_df = pol_df[pol_df["iso3"].isin(vac_iso_codes)]

#checking number of countries
pol_df[['iso3']].nunique()

iso3    173
dtype: int64

In [51]:
#checking data types
pol_df.dtypes

iso3             object
year              int64
polarization    float64
dtype: object

In [52]:
#checking shape
pol_df.shape

(1903, 3)

In [53]:
#dropping empty rows and checking if any were dropped
pol_df = pol_df.dropna()
pol_df.shape

(1903, 3)

In [54]:
#getting standardized country names
pol_df = pol_df.merge(countries_map, on='iso3', how='left')

#reordering columns
pol_df = pol_df[['country', 'iso3', 'year', 'polarization']]
pol_df.head()

,country,iso3,year,polarization
0,Mexico,MEX,2012,1.0
1,Mexico,MEX,2013,1.0
2,Mexico,MEX,2014,1.0
3,Mexico,MEX,2015,1.0
4,Mexico,MEX,2016,1.0


In [55]:
#checking values of polarization as first 5 were '1.0' only
pol_df['polarization'].value_counts()

polarization
 1.43    29
 1.58    24
 1.05    24
 2.03    23
 0.74    23
         ..
-0.29     1
-2.66     1
-1.54     1
-1.69     1
 2.43     1
Name: count, Length: 362, dtype: int64

### Trust - World Values Survey

World Values Survey is a global research project that explores people's values and beliefs, how they change over time, and what social and political impact they have. For this project, data regarding trust (in people, press, government, universities, big companies and who) will be extracted.

It includes individual responses on participants, so average country scores will be calculated during data clean-up. 

In [ ]:
#reading the data
wvs = pd.read_csv('06_WVS7.csv', low_memory=False)
wvs.head()

In [ ]:
#choosing applicable columns
wvs_df = wvs[['B_COUNTRY_ALPHA', 'A_YEAR', 'Q57', 'Q64', 'Q66', 'Q71', 'Q75', 'Q77', 'Q88']]

#renaming columns based on the codebook
wvs_df = wvs_df.set_axis(['iso3', 'year', 'poeple_trust', 'church_trust', 'press_trust', 'gov_trust', 'uni_trust', 'comp_trust', 'who_trust'], axis=1)

In [ ]:
wvs_df.dtypes

In [ ]:
#replacing wvs missing values codes (-) with NaN
wvs_df.replace([-1, -2, -4, -5], np.NaN, inplace=True)

#calculating missing values
wvs_df.isna().sum()

In [ ]:
wvs_df.shape

In [ ]:
#calculating mean values for the variables
wvs_df = wvs_df.groupby(['iso3', 'year']).mean().round(2).reset_index()
wvs_df.head()

Data in WVS data for different countries were collected for different years. As COVID-19 could impact trust values, I'm calculating the number of countries where data was collected pre- and post-covid:

In [ ]:
pre_cov_wvs = wvs_df[wvs_df['year'] < 2020]
len(pre_cov_wvs)

In [ ]:
post_cov_wvs = wvs_df[wvs_df['year'] > 2019]
len(post_cov_wvs)

*Post-exploratory analysis note: there is no direct link between the trust and covid_19. It might be moderated by a different variable but anyways the data will the variables from the survey will be included in the trends analysis (no trends over time as data for different years is not available), as insights might be valuable.*

In [ ]:
wvs_df = wvs_df.drop(['year'], axis=1)

### Geo Data - USND

For regional analysis data for regions and sub-regions standardized by USND.

In [ ]:
#reading the data
geo = pd.read_csv('09_UNSD.csv', sep=';')
geo.head()

In [ ]:
#choosing applicable columns
geo_df = geo[['Region Name', 'Sub-region Name', 'ISO-alpha3 Code']]

#renaming columns
geo_df = geo_df.set_axis(['region', 'subregion', 'iso3'], axis=1)
geo_df.head()

## Yearly Data Merge

In [ ]:
# Changing wide tables to long tables
v_index_long = v_index_df.melt(id_vars=['country', 'iso3'], var_name='year', value_name='vac_index')
gdp_long = gdp_df.melt(id_vars=['country', 'iso3'], var_name='year', value_name='gdp')
health_long = health_df.melt(id_vars=['country', 'iso3'], var_name='year', value_name='health_exp')
child_mort_long = child_mort_df.melt(id_vars=['country', 'iso3'], var_name='year', value_name='child_mort')
internet_long = internet_df.melt(id_vars=['country', 'iso3'], var_name='year', value_name='internet_use')

#changing all year values to integer
for df in [v_index_long, gdp_long, health_long, child_mort_long, internet_long, trust_df, pol_df]:
    df['year'] = df['year'].astype(int)

# Merging with country, iso3, year keys including values matching target v_index only
from functools import reduce
dfs_to_merge = [v_index_long, gdp_long, health_long, child_mort_long, internet_long, trust_df, pol_df]
df = reduce(lambda left, right: pd.merge(left, right, on=['country', 'iso3', 'year'], how='left'), dfs_to_merge)

#merging with geo data on iso3
df = df.merge(geo_df, on='iso3', how='left')

# Converting year to int type
df['year'] = df['year'].astype(int)

#reordering columns
df = df[['country', 'region', 'subregion'] + [col for col in df.columns if col not in ['country', 'region', 'subregion']]]

df.head()

Checking what countries are missing for each variable (except gov_trust as this is bonus only with oecd countries):

In [ ]:
# List of columns to check
cols_to_check = ['gdp', 'health_exp', 'child_mort', 'internet_use', 'polarization']

# Dictionary to store countries with missing values per column
missing_by_column = {}

for col in cols_to_check:
    missing_countries = df[df[col].isna()]['country'].unique()
    missing_by_column[col] = missing_countries

# Print the results
for col, countries in missing_by_column.items():
    print(f"\nMissing in '{col}': ({len(countries)} countries)")
    print(sorted(countries))

Data for polarization is not crucial for the analysis - the variable will be rather used as a moderator. Thus, for the final analysis only countries missing in gdp, health_exp, child_mort and internet_use will be excluded.

In [ ]:
# Creating a list of countries with any missing values in selected columns
countries_with_missing = df[df[['gdp', 'health_exp', 'child_mort', 'internet_use']].isna().any(axis=1)]['country'].unique()

# Removing those countries
df_cleaned = df[~df['country'].isin(countries_with_missing)].copy()

#final numer of countries
print(f"Number of countries: {df_cleaned['country'].nunique()}")

This is final DataFrame for analysis exploratory analysis and predictions.

In [ ]:
df_cleaned.to_csv("00_Immunization_db.csv", index=False)